<a href="https://colab.research.google.com/github/meti-94/OpenQA/blob/main/Evaluation_Whole_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing necessary libraries, some of them would be used in the underlying script 

In [1]:
!pip install transformers -q
!pip install fuzzywuzzy -q
!pip install python-Levenshtein -q
!pip install pattern -q

     |████████████████████████████████| 3.4 MB 4.8 MB/s 
     |████████████████████████████████| 61 kB 192 kB/s 
     |████████████████████████████████| 596 kB 34.8 MB/s 
     |████████████████████████████████| 3.3 MB 34.4 MB/s 
     |████████████████████████████████| 895 kB 32.6 MB/s 
     |████████████████████████████████| 50 kB 2.5 MB/s 
     |████████████████████████████████| 22.2 MB 1.9 MB/s 
     |████████████████████████████████| 87 kB 5.9 MB/s 
     |████████████████████████████████| 81 kB 7.9 MB/s 
     |████████████████████████████████| 5.6 MB 40.1 MB/s 
     |████████████████████████████████| 5.6 MB 40.5 MB/s 
     |████████████████████████████████| 419 kB 46.3 MB/s 
     |████████████████████████████████| 97 kB 5.6 MB/s 
     |████████████████████████████████| 3.6 MB 36.0 MB/s 


In this section we clone our repository and run the script for create Entity Linking candidates file. Default values for all the paths are pointing to the files which we already stored on google drive. Resulting file would be available after execution of the script with the name Linking.xlsx

In [3]:
!git clone https://github.com/meti-94/OpenQA.git
!python ./OpenQA/src/EntityLinking.py 

Cloning into 'OpenQA'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (188/188), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 188 (delta 70), reused 119 (delta 26), pack-reused 0
Receiving objects: 100% (188/188), 89.08 MiB | 13.81 MiB/s, done.
Resolving deltas: 100% (70/70), done.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Reverb Questions Count: 5003	Freebase Questions Count: 21619
100% 407267/407267 [01:03<00:00, 6450.00it/s]

Total MIDs before augmentation: 1951909	Unmatched (Added) MIDs: 669962	 Matched MIDs: 144572
Total Entity Strings before augmentation: 17555942	Added Entity Strings: 138167
Total type of text: 4936783
Max Length of entry is 249717, text is [('fb:m.01brl8f', "concerto no. 4 in f minor , op. 8 , rv 297 , `` l'inverno '' ( winter ) : iii. allegro", 'fb:type.object.name'), ('fb:m.08crqb', 'kisling , jérémie', 'fb:common.topic.alias'), ('fb:m.0gbl

After creating the named entity candidate, we are going to create candidates for relations. This file contains the first freebase 10 choices and reverb candidates. 

In [5]:
!rm -rf ./results
!python ./OpenQA/src/RelationDetection.py

Streaming output truncated to the last 5000 lines.
Predicting ...:  81% 21624/26622 [1:29:40<21:54,  3.80it/s]
Predicting ...:  81% 21625/26622 [1:29:40<21:11,  3.93it/s]
Predicting ...:  81% 21626/26622 [1:29:40<22:01,  3.78it/s]
Predicting ...:  81% 21627/26622 [1:29:41<23:03,  3.61it/s]
Predicting ...:  81% 21628/26622 [1:29:41<23:20,  3.57it/s]
Predicting ...:  81% 21629/26622 [1:29:41<22:45,  3.66it/s]
Predicting ...:  81% 21630/26622 [1:29:41<22:10,  3.75it/s]
Predicting ...:  81% 21631/26622 [1:29:42<22:40,  3.67it/s]
Predicting ...:  81% 21632/26622 [1:29:42<21:30,  3.87it/s]
Predicting ...:  81% 21633/26622 [1:29:42<21:16,  3.91it/s]
Predicting ...:  81% 21634/26622 [1:29:42<20:35,  4.04it/s]
Predicting ...:  81% 21635/26622 [1:29:43<21:05,  3.94it/s]
Predicting ...:  81% 21636/26622 [1:29:43<20:47,  4.00it/s]
Predicting ...:  81% 21637/26622 [1:29:43<21:07,  3.93it/s]
Predicting ...:  81% 21638/26622 [1:29:43<20:34,  4.04it/s]
Predicting ...:  81% 21639/26622 [1:29:44<21:13, 

Importing necessary libraries 

In [7]:
import pandas as pd
import pickle
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()

In [8]:
# reading the file which contains the Entity Linking candidates for each question
entity_df = pd.read_excel('./Linking.xlsx')
# reading the file which contains the relation candidates for each question
relation_df = pd.read_excel('./dbg.xlsx')
# merging the relation and entity linking files
predictions = pd.merge(entity_df, relation_df, on='Question').drop_duplicates()
# reading reverb test portion, Question [question string] Triple [arg1, rel, arg2] and Reverb_no [which line the question was designed for]
reverb_df = pd.read_excel('/content/drive/MyDrive/data_freebase/test.xlsx')[['Question', 'triple', 'Reverb_no']]
reverb_df['Relation'] = reverb_df['triple'].apply(lambda item:eval(item)[1])
reverb_df = reverb_df[['Question', 'Relation', 'Reverb_no']]
# reading the freebase test portion and process it into the ground truth data like what we had earlier for reverb
freebase_df = pd.read_excel('/content/drive/MyDrive/data_freebase/test_useful_records.xlsx')[['Question', 'relation_type']]
freebase_df['Relation'] = freebase_df['relation_type']
freebase_df['Reverb_no'] = freebase_df['relation_type'].apply(lambda item:'')
freebase_df = freebase_df[['Question', 'Relation', 'Reverb_no']]
# concatenating all grand truth data into a one object which must named actual based on machine learning methodology 
actuals = pd.concat([reverb_df, freebase_df])

In [9]:
data = pd.merge(actuals, predictions, on='Question', how='inner')
data.head()

,Question,Relation,Reverb_no,NER,Answer,HIT@,Candidates,Freebase,Reverb
0,how do Protocols vary,vary by,267285,protocols,5f5d3e85-c87f-4fa2-b636-5ce2d89acab5,10,"[(('b31be1a0-0c38-4227-bdd0-5a78f2196253', 'pr...","[('fb:medicine.disease.medical_specialties', 0...","{""do n't vary based on"": 0.84525030966336, 'do..."
1,of what is A Shooting Star another brand,is another brand of,8600,a shooting star,254a68ac-99ca-4a53-8d2a-b841ac0fbbe2,3,"[(('305bf346-b11f-4cd0-8f21-f9e58ea327c1', 'a ...","[('fb:automotive.model.successor', 0.043253843...","{'is another brand of': 1.0000000000000002, 'i..."
2,where was the hometown of Angie Debo,was the hometown of,212931,angie debo,baf4dcad-f15d-40ea-9517-5254ab996785,3,"[(('fb:m.05c0fr7', 'angie debo', 'fb:type.obje...","[('fb:music.artist.origin', 0.861007), ('fb:pe...","{'was the hometown of': 1.0000000000000002, 'i..."
3,where Curium does not occur,does not occur in,89850,curium,fb:m.025tkv0,1,"[(('fb:m.025tkv0', 'curium', 'fb:type.object.n...",[('fb:theater.theater_production.performed_at'...,"{'does not occur in': 0.986554414128868, 'does..."
4,who died in Moraga,died in,385331,moraga,a1fd5fa8-d7db-4a01-877d-7f6a336b768e,3,"[(('fb:m.0qysj', 'moraga', 'fb:type.object.nam...","[('fb:location.location.people_born_here', 0.8...","{'died in': 1.0000000000000002, 'Died in': 1.0..."


Reading reverb to freebase mapping which we created earlier for merging MIDs

In [10]:
reverb2freebace = pd.read_csv('/content/drive/MyDrive/reverb2freebase.csv')
reverb2freebace['freebase_ID_argument1'] = reverb2freebace['freebase_ID_argument1'].apply(lambda string:'fb:m.'+str(string))
reverb2freebace['conf'] = reverb2freebace['conf'].astype(float)

Reading indices from Freebase KB. They were created using BBQA repo create_index.sh script :)

In [11]:
# mapping between MIDs and names in the form of dict['MID']=['str1', 'str2', ...,  'strN']
with open('/content/drive/MyDrive/indexes/names_2M.pkl', 'rb') as f:
    mid2name = pickle.load(f)
# mapping between MIDs and Relations in the form of dict['MID']=[{'fb:common.topic.notable_types', 'fb:people.person.gender', 'fb:people.person.profession'}]
with open('/content/drive/MyDrive/indexes/reachability_2M.pkl', 'rb') as f:
    reachability_2M = pickle.load(f)

In this section we are combining relation from reverb with what we already have in KB (Freebase). Note: for reverb's relation the Reverb_no is the label 

In [12]:
def combine(reachability_2M, reverb2freebace):
  for index, row in tqdm(reverb2freebace.iterrows(), total=reverb2freebace.shape[0], desc='Combining Relations ... '):
    if row['freebase_ID_argument1'] in mid2name:
      mid1 = mid = row['freebase_ID_argument1'] 
    else:
      mid1 = mid = row['argument1_uuid']
    mid2 = mid = row['argument2_uuid']
    relation = row['rel']
    reverb_no = row['reverb_no']
    try:
      temp = reachability_2M[mid1]
    except:
      reachability_2M[mid1] = set()
      temp = reachability_2M[mid1]
    temp.add((relation, reverb_no))
    try:
      temp = reachability_2M[mid2]
    except:
      reachability_2M[mid2] = set()
      temp = reachability_2M[mid2]
    temp.add((relation, reverb_no))
  return reachability_2M


In [13]:
reachability_2M = combine(reachability_2M, reverb2freebace)

Combining Relations ... :   0%|          | 0/407267 [00:00<?, ?it/s]

This part is responsible for creating final candidates for each question. first of all we do perform Cartesian product on e member of E and r member of R then prune the falty ordered pairs 

In [14]:
def create_candidates(row):
  ner = row['Candidates']
  freebase = row['Freebase']
  reverb = row['Reverb']
  candidates = []
  rvb = [(key, value) for key, value in eval(reverb).items()]
  rels = eval(freebase)+rvb
  try:
    for idx1, ((mid, string, conf), sim1) in enumerate(eval(ner)):
        relations = list(reachability_2M[mid])
        for relation in relations:
          if isinstance(relation, tuple):
            for idx2, (rel, sim2) in enumerate(rels):
              if relation[0]==rel:
                candidates.append((mid, rel, sim1, sim2, conf, relation[1]))
          else:
            for idx2, (rel, sim2) in enumerate(rels):
              if relation==rel:
                candidates.append((mid, rel, sim1, sim2, conf))
    return sorted(candidates, key=lambda item:item[2]+item[3], reverse=True)
  except:
    return None

In [15]:
data['answers'] = data.progress_apply(create_candidates, axis=1)

  0%|          | 0/26648 [00:00<?, ?it/s]

In [16]:
data = data.dropna(subset=['answers'])
data.answers.isnull().sum()

0

Final Evaluation 

In [17]:
top1 = 0
top3 = 0
top5 = 0
top10 = 0
top20 = 0
top50 = 0
top100 = 0
for idx1, row in data.iterrows():
  for idx2, candidate in enumerate(row['answers']):
    if len(candidate)==6:
      if candidate[-1]==row['Reverb_no']:
        if idx2 in range(1):
            # print(candidate, row['Reverb_no'])
            top1 += 1
        if idx2 in range(3):
            top3 += 1
        if idx2 in range(5):
            top5 += 1
        if idx2 in range(10):
            top10 += 1
        if idx2 in range(20):
            top20 += 1
        if idx2 in range(50):
            top50 += 1
        if idx2 in range(100):
            top100 += 1
            break 
    else:
      if (candidate[0]==row['Answer']) and (candidate[1]==row['Relation']):
        if idx2 in range(1):
            # print(candidate, row['Answer'], row['Relation'])
            top1 += 1
        if idx2 in range(3):
            top3 += 1
        if idx2 in range(5):
            top5 += 1
        if idx2 in range(10):
            top10 += 1
        if idx2 in range(20):
            top20 += 1
        if idx2 in range(50):
            top50 += 1
        if idx2 in range(100):
            top100 += 1
            break 
   

# print(data_type)
total = len(data)
print("Top1 Answers: {}".format(top1 / total))
print("Top3 Answers: {}".format(top3 / total))
print("Top5 Answers: {}".format(top5 / total))
print("Top10 Answers: {}".format(top10 / total))
print("Top20 Answers: {}".format(top20 / total))
print("Top50 Answers: {}".format(top50 / total))
print("Top100 Answers: {}".format(top100 / total))

Top1 Answers: 0.7106507543346093
Top3 Answers: 0.791225699917436
Top5 Answers: 0.8100653006079712
Top10 Answers: 0.8271410343015837
Top20 Answers: 0.8374240036027921
Top50 Answers: 0.8450048787810553
Top100 Answers: 0.8472190947984688


In [ ]:
len(data)

26646

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
